### Import libraries

In [ ]:
from complexity.airspace import airspace
from traffic.data import nm_airspaces
from utils import viz as viz

### Defining airspace

In [ ]:
# Deinition of airspace
lmmm = airspace(
    id="LMMM",
    volume = nm_airspaces['LMMMALL']
)

### Data preparation

#### Data fetching

In [ ]:
# Fetching of ADS-B data
lmmm.data_fetch(
    start_date="2019-01-01",
    end_date="2020-01-01",
)

#### Data pre-processing

In [ ]:
# Preprocessing ADS-B data
lmmm.data_preprocess()

In [ ]:
# # Visualisation of airspace and trajectories before pre-processing
# fig1 = lsaguac.plot(traj_sample= True, traj_num=10, reduced=False)
# fig1.show()
# # Visualisation of airspace and trajectories after pre-processing
# fig2 = lsaguac.plot(traj_sample= True, traj_num=10, reduced=True)
# fig2.show()

### Identification and extraction of low-traffic trajectories

#### Measuring hourly traffic load

In [ ]:
# Generation of dataframe containing hourly entry counts
lmmm.hourly_generate_df()
# Plotting of heatmap of entry counts
fig = lmmm.hourly_heatmap()
fig.show()

#### Definition of low-traffic threshold

In [ ]:
# Generation of heatmap-like plot
fig1 = lmmm.hourly_heatmap_low(reference_type='max_perc', reference_value=0.45)
fig1.show()
# Generation of multiple boxplots
fig2 = lmmm.hourly_boxplots(reference_type='max_perc', reference_value=0.45)
fig2.show()
# Generation of Cumulative distribution function
fig3 = lmmm.hourly_cdf(reference_type='max_perc', reference_value=0.45)
fig3.show()

#### Extracting trajectories

In [ ]:
lmmm.reduce_low_traffic(reference_type='max_perc', reference_value=0.45)

### Monte Carlo simulations

In [ ]:
temp = lsaguac.run_simulation(duration=24, interval=10, start_time='2019-01-01 00:00:00')

In [ ]:
temp.flight_id.unique().shape

In [ ]:
temp.timestamp.max()

In [ ]:
shower = temp[temp.flight_id == 'MSR801_110_15:31:20']

In [ ]:
import pandas as pd
temp2 = temp[temp.timestamp < pd.Timestamp('2019-01-02 00:00:00')]

In [ ]:
x = pd.Timestamp('2019-01-01 00:00:00')

In [ ]:
y = x + pd.Timedelta(hours=24)
y

In [ ]:
temp2.flight_id.unique().shape

In [ ]:
import matplotlib.pyplot as plt

# plot histogram of temp.timestamp
fig = plt.figure(figsize=(10, 5))
plt.hist(temp.timestamp, bins=365)
plt.show()

In [ ]:
temp.max()

In [ ]:
temp.flight_id.unique().shape

In [ ]:
temp.timestamp.max()

In [ ]:
lsaguac.generate_cells(dim=5, alt_diff=1000)
# lsaguac.visualise_cells()
lsaguac.parallel_simulation_run(duration = 24,
                                interval = 20,
                                runs = 10000,
                                max_process = 50,
                                start_num = 0)

In [ ]:
# pickle load file into variable temp
temp = pickle.load(open('/cluster/home/krum/github/VT2_airspace_complexity/data/LSAGUAC/08_monte_carlo/24_20/cubes/9408_results.pkl', 'rb'))


In [ ]:
from utils import general as util_general
from traffic.core import Traffic
import pandas as pd
import numpy as np

home_path = util_general.get_project_root()

num = 10 # number of simulations (monte carlo)
duration = 24
interval = 20
start_time = pd.Timestamp("2000-01-01 00:00:00")

# Load data and get ids and dataframe
trajs_low = Traffic.from_file(
    f"{home_path}/data/LSAGUAC/05_low_traffic/trajs_tma_low.parquet"
)
ids = trajs_low.flight_ids
trajs_low_data = trajs_low.data

totalseconds = duration * 60 * 60
amount_deploys = int(totalseconds / interval)

timelist = []
timer = 0
for i in range(int(amount_deploys)):
    timelist.append(start_time + pd.Timedelta(seconds=timer))
    timer = timer + interval
times = np.array(timelist)

indices = np.random.default_rng().choice(
    len(ids), len(timelist), replace=True
)
ids = np.array(ids)[indices]

grouped = trajs_low_data.groupby("flight_id")

# generate simulated trajectories
# print("Generating simulated trajectories...")
df_all = []

for id, tm in zip(ids, times):
    # traj_time = start_time + pd.Timedelta(seconds=tm)
    temp = grouped.get_group(id)
    timedelta = temp["timestamp"] - temp["timestamp"].iloc[0]
    new_timestamp = tm + timedelta
    timestring = str(new_timestamp.iloc[0].time())
    flight_id = temp["flight_id"].iloc[0]
    temp = temp[["latitude", "longitude", "altitude", "icao24"]]
    temp.insert(0, "timestamp", new_timestamp)
    temp.insert(1, "flight_id", flight_id + "_" + timestring)
    df_all.append(temp)

df_traf = (
    pd.concat(df_all, axis=0)
    .sort_values(by=["timestamp"])
    .reset_index()
)

In [ ]:
timelist[-1]

In [ ]:
len(indices)

In [ ]:
temp

In [ ]:
lsaguac.visualise_cells()

In [ ]:
lsaguac

In [ ]:
# lsaguac.run_simulation(1, 10)

In [ ]:
lsaguac.parallel_simulation_run(duration = 24,
                                interval = 20,
                                runs = 10000,
                                max_process = 50,
                                start_num = 0)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from traffic.core import Traffic
from utils import general as util_general

In [ ]:
import pickle
import glob

home_path = util_general.get_project_root()

# get list of all files in the folder
files = glob.glob(f"{home_path}/data/LSAGUAC/08_monte_carlo/24_120/total/*.pkl", recursive=True)

values = []

for file in files:
    with open(file, 'rb') as f:
        x = pickle.load(f)
        values.append(x)

In [ ]:
len(values)

In [ ]:
# reduce values to only contain  entries below 50
values = [x for x in values if x < 50]

In [ ]:
len(values)

In [ ]:
# histogram of values
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(values, bins=100)

# x axis label
plt.xlabel('Number of occurences')

# plot title
plt.title('Histogram of number of total occurences')

plt.show()

# get 95% confidence interval
from scipy import stats

stats.norm.interval(0.90, loc=np.mean(values), scale=np.std(values))




In [ ]:
# get max value of values
min(values)

In [ ]:
home_path = util_general.get_project_root()

trajs_low = Traffic.from_file(
            f"{home_path}/data/LSAGUAC/05_low_traffic/trajs_tma_low.parquet"
        )

In [ ]:
duration = 1
interval = 180

ids = trajs_low.flight_ids
trajs_low_data = trajs_low.data

totalseconds = duration * 24 * 60 * 60
amount_deploys = int(totalseconds / interval)

timelist = []

timer = 0
for i in range(int(amount_deploys)):
    timelist.append(timer)
    timer = timer + interval

indices = np.random.default_rng().choice(
    len(ids), len(timelist), replace=True
)

random_ids = np.array(ids)[indices]
random_times = np.array(timelist)

grouped = trajs_low_data.groupby("flight_id")

# generate simulated trajectories
print("Generating simulated trajectories...")
df_all = []
start_time = pd.Timestamp("2000-01-01 00:00:00")

for id, tm in tqdm(
    zip(random_ids, random_times), total=len(random_ids)
):
    traj_time = start_time + pd.Timedelta(seconds=tm)
    temp = grouped.get_group(id)
    timedelta = temp["timestamp"] - temp["timestamp"].iloc[0]
    new_timestamp = traj_time + timedelta
    timestring = str(new_timestamp.iloc[0].time())
    flight_id = temp["flight_id"].iloc[0]
    temp = temp[["latitude", "longitude", "altitude", "icao24"]]
    temp.insert(0, "timestamp", new_timestamp)
    temp.insert(1, "flight_id", flight_id + "_" + timestring)
    df_all.append(temp)

df_traf = (
    pd.concat(df_all, axis=0)
    .sort_values(by=["timestamp"])
    .reset_index()
)

In [ ]:
from utils import general as util_general
from traffic.core import Traffic

home_path = util_general.get_project_root()

duration = 1
interval = 120
amount = 100

trajs_low = Traffic.from_file(
            f"{home_path}/data/LSAGUAC/05_low_traffic/trajs_tma_low.parquet"
        )

durations = [duration for i in range(amount)]
intervals = [interval for i in range(amount)]
trajectories = [trajs_low for i in range(amount)]

t = [
    (duras, intes, trajs)
    for duras, intes, trajs in zip(durations, intervals, trajectories)
]


In [ ]:
t[0][2]

In [ ]:
import glob
import pickle

# # Set the path to the folder containing the pkl files
# folder_path = '/cluster/home/krum/github/VT2_airspace_complexity/data/LSAGUAC/07_cube_data/'

# # Define the pattern to match the pkl files (i.e., files starting with "range_18000")
# pattern = f"{folder_path}/range_24000*.pkl"

# # Load the pkl files matching the pattern into a list
# pkl_files = []
# for file_path in glob.glob(pattern):
#     with open(file_path, 'rb') as f:
#         pkl_files.append(pickle.load(f))

In [ ]:
lat_min = []
lat_max = []
lon_min = []
lon_max = []
alt_min = []
alt_max = []

count = []

for cube in lsaguac.cubes:
    lat_max.append(cube.lat_max)
    lat_min.append(cube.lat_min)
    lon_max.append(cube.lon_max)
    lon_min.append(cube.lon_min)
    alt_max.append(cube.alt_high)
    alt_min.append(cube.alt_low)

    with open(f'/cluster/home/krum/github/VT2_airspace_complexity/data/LSAGUAC/07_cube_data/{cube.id}_pairs.pkl', 'rb') as f:
        count.append(len(pickle.load(f)))



In [ ]:
import pandas as pd
df = pd.DataFrame(
    list(zip(lat_min, lat_max, lon_min, lon_max, alt_min, alt_max, count)),
    columns=['lat_min', 'lat_max', 'lon_min', 'lon_max', 'alt_min', 'alt_max', 'count']
)
# get total of count column
print(df['count'].sum())

df = df[df.alt_min == 36000]

In [ ]:
# # read data/LSGACU/06_simulation/trajs_simulation.parquet
# import pandas as pd
# df = pd.read_parquet('/cluster/home/krum/github/VT2_airspace_complexity/data/LSAGUAC/06_simulation/trajs_simulation.parquet')
# df.flight_id.nunique()

In [ ]:
# sort df by count
df.sort_values(by=['count'], ascending=False)

In [ ]:
# nm_airspaces['LSAGUAC'].centroid.coords[0]

In [ ]:
df['count'].max()

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import Normalize

fig = go.Figure(go.Scattermapbox())
fig.update_layout(
    mapbox_style="mapbox://styles/jakrum/clgqc6e8u00it01qzgtb4gg1z",
    mapbox_accesstoken='pk.eyJ1IjoiamFrcnVtIiwiYSI6ImNsZ3FjM3BiMzA3dzYzZHMzNHRkZnFtb3EifQ.ydDFlmylEcRCkRLWXqL1Cg',
    showlegend=False,
    height=1000,
    width=1000,
    margin={"l": 0, "b": 0, "t": 0, "r": 0},
    mapbox_center_lat=nm_airspaces['LSAGUAC'].centroid.coords[0][1],
    mapbox_center_lon=nm_airspaces['LSAGUAC'].centroid.coords[0][0],
    mapbox_zoom=7,
    # tickmode="array",
    # showticklabels=False,
)

# cmap = plt.cm.get_cmap('Blues')
# norm = mcolors.Normalize(vmin=df['count'].min(), vmax=df['count'].max())
cmap = cm.ScalarMappable(norm=Normalize(vmin=df['count'].min(), vmax=df['count'].max()), cmap='plasma')

def colorscale(value):
    color = cmap.to_rgba(value)
    color = [int(c * 255) for c in color]
    color_hex = '#{:02x}{:02x}{:02x}'.format(*color[:3])
    return color_hex

for row in df.iterrows():
    fig.add_trace(
        go.Scattermapbox(
            lat=[row[1][0], row[1][0], row[1][1], row[1][1], row[1][0]],
            lon=[row[1][2], row[1][3], row[1][3], row[1][2], row[1][2]],
            mode="lines",
            line=dict(width=2, color='black'),
            fill="toself",
            fillcolor=colorscale(row[1][6]),
            text=f"count: {row[1][6]}",
            opacity=0.2,
            name="Rectangle",
        )
    )
        # calculate center point of rectangle
    center_lat = (row[1][0] + row[1][1]) / 2
    center_lon = (row[1][2] + row[1][3]) / 2
    # add count as text at center point
    fig.add_trace(
        go.Scattermapbox(
            lat=[center_lat],
            lon=[center_lon],
            mode="text",
            text=[int(row[1][6])],
            textfont=dict(size=8, color='grey'),
            textposition="middle center",
            showlegend=False,
        )
    )

# Add airspace shape to the plot
lons, lats = nm_airspaces['LSAGUAC'].shape.exterior.xy
trace = go.Scattermapbox(
    mode="lines",
    lat=list(lats),
    lon=list(lons),
    line=dict(width=2, color="red"),
)

colorbar_trace  = go.Scatter(x=[None],
                             y=[None],
                             mode='markers',
                             marker=dict(
                                 colorscale='plasma', 
                                 showscale=True,
                                 cmin=df['count'].min(),
                                 cmax=df['count'].max(),
                                 colorbar=dict(thickness=20, outlinewidth=0, title='occurences')
                             ),
                             hoverinfo='none'
                            )

fig['layout']['showlegend'] = False
fig.add_trace(colorbar_trace)

# add figure title
fig.update_layout(
    title_text="Altitude level 36'000ft - 37'000ft",
    title_x=0.5,
    margin={"l": 0, "b": 0, "t": 40, "r": 0},
)

fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.add_trace(trace)
fig.show()

In [ ]:
row[1][6]

In [ ]:
import folium
from folium import GeoJson
import pandas as pd
from branca.colormap import linear

# Create a folium map centered on the middle of the bounding box of the rectangles
center_lat = (df['lat_min'].min() + df['lat_max'].max()) / 2
center_lon = (df['lon_min'].min() + df['lon_max'].max()) / 2
# m = folium.Map(location=[center_lat, center_lon], zoom_start=5)
# m = folium.Map(location=[center_lat, center_lon], zoom_start=5, tiles='Stamen Toner Lite')
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=5,
    tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}.png',
    attr='Map data &copy; <a href="https://stamen.com/">Stamen Design</a>'
)

colorscale = linear.YlOrRd_04.scale(df['count'].min(), df['count'].max())

# Define a function to create a rectangle marker for each row in the dataframe
def create_marker(row):
    # Calculate the coordinates of the rectangle corners
    lat1, lon1, lat2, lon2 = row['lat_min'], row['lon_min'], row['lat_max'], row['lon_max']
    # Calculate the center of the rectangle
    center_lat, center_lon = (lat1 + lat2) / 2, (lon1 + lon2) / 2
    # Create a folium rectangle marker with the count as its color
    folium.Rectangle(
        bounds=[(lat1, lon1), (lat2, lon2)],
        fill_color=colorscale(row['count']),  # set the color based on the count
        # fill_color=f'#{int(row["count"]):02x}0000'
        fill_opacity=0.8,
        color='black',
        weight=0.5,
        popup=f'Count: {row["count"]}',
        tooltip=f'Count: {row["count"]}',
        radius=0
    ).add_to(m)

# Apply the create_marker function to each row in the dataframe
df.apply(create_marker, axis=1)

# Show the map
m

In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Define the layout of the map
layout = go.Layout(
    mapbox=dict(
        center=dict(lat=center_lat, lon=center_lon),
        style="carto-positron",
        zoom=5
    ),
    margin=dict(l=0, r=0, t=0, b=0)
)

# Define the data for the rectangles
data = [
    go.Scattermapbox(
        lat=[(row['lat_min'] + row['lat_max']) / 2 for _, row in df.iterrows()],
        lon=[(row['lon_min'] + row['lon_max']) / 2 for _, row in df.iterrows()],
        mode='markers',
        marker=dict(
            color=df['count'],
            colorscale='YlOrRd',
            cmin=df['count'].min(),
            cmax=df['count'].max(),
            opacity=0.8,
            sizemode='area',
            # sizeref=0.1,
            # sizemin=2,
            # sizemax=20,
            # line=dict(
            #     color='black',
            #     width=0.5
            # )
        ),
        hovertemplate='Count: %{marker.color}<extra></extra>',
        name='Rectangles'
    )
]

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()

### Cell counts

In [ ]:
from traffic.core import Traffic
import pandas as pd
import numpy as np
from tqdm.auto import tqdmx
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
trajs_low = Traffic.from_file("../data/LSAGUAC/05_low_traffic/trajs_tma_low.parquet")
ids = trajs_low.flight_ids
trajs_low_data = trajs_low.data

In [ ]:
days = 10
freq = 5

seconds = days * 24 * 60 * 60

amount_deploys = seconds / freq

timelist = []

timer = 0
for i in range(int(amount_deploys)):
    timelist.append(timer)
    timer = timer + freq

indices = np.random.default_rng().choice(len(ids), len(timelist), replace=True)

random_ids = np.array(ids)[indices]
random_times = np.array(timelist)

grouped = trajs_low_data.groupby('flight_id')


In [ ]:
df_all = []

start_time = pd.Timestamp("2000-01-01 00:00:00")

for id, tm in tqdm(zip(random_ids, random_times), total=len(random_ids)):
    traj_time = start_time + pd.Timedelta(seconds=tm)
    temp = grouped.get_group(id)
    timedelta = temp["timestamp"] - temp["timestamp"].iloc[0]
    new_timestamp = traj_time + timedelta
    timestring = str(new_timestamp.iloc[0].time())
    flight_id = temp['flight_id'].iloc[0]
    temp = temp[['latitude', 'longitude', 'altitude', 'icao24']]
    temp.insert(0, "timestamp", new_timestamp)
    temp.insert(1, "flight_id", flight_id + "_" + timestring)
    df_all.append(temp)

In [ ]:
df_traf = pd.concat(df_all, axis=0).sort_values(by=['timestamp']).reset_index()
df_traf.to_parquet(f"../data/LSAGUAC/06_cube_data/simulation_trajs.parquet", index=False)

In [ ]:
import pickle

for cube in tqdm(lsaguac.cubes):
    subset = df_traf.loc[(df_traf['latitude'] >= cube.lat_min) & 
                     (df_traf['latitude'] <= cube.lat_max) &
                     (df_traf['longitude'] >= cube.lon_min) &
                     (df_traf['longitude'] <= cube.lon_max) &
                     (df_traf['altitude'] >= cube.alt_low) &
                     (df_traf['altitude'] <= cube.alt_high)]
    subset.to_parquet(f"../data/LSAGUAC/06_cube_data/{cube.id}_trajs.parquet", index=False)

    in_out = (
        subset.groupby("flight_id")["timestamp"]
        .agg(["min", "max"])
        .reset_index()
        .sort_values(by="min")
        )
    in_out.to_parquet(f"../data/LSAGUAC/06_cube_data/{cube.id}_inout.parquet", index=False)

    df = in_out

    count = 0
    pairs = []

    # Create a dictionary of flights indexed by their max value
    flight_dict = {flight.max: [flight.flight_id] for flight in df.itertuples()}

    # Find overlapping flights
    pairs = []
    count = 0
    for flight in df.itertuples():
        matches = []
        for other_flight in flight_dict.get(flight.min, []):
            if flight.max > df.loc[df["flight_id"] == other_flight, "min"].values[0]:
                matches.append(other_flight)
                count += 1
        if matches:
            matches.append(flight.flight_id)
            pairs.append(tuple(matches))

    with open(f"../data/LSAGUAC/06_cube_data/{cube.id}_pairs.pkl", "wb") as fp:
        pickle.dump(pairs, fp)

    print(f"{cube.id} with {len(df)} flights with {len(pairs)} overlapping intervals.")

In [ ]:
import pickle

with open(f"../data/LSAGUAC/07_cube_data/range_24000_grid_59_pairs.pkl", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

b

In [ ]:
import pandas as pd

df_all = pd.read_parquet("../data/LSAGUAC/06_simulation/trajs_simulation.parquet")
df_cube = pd.read_parquet("../data/LSAGUAC/07_cube_data/range_24000_grid_59_trajs.parquet")

In [ ]:
import plotly.graph_objects as go

id1 = 'EJU54ZH_37497_15:52:05'
id2 = 'BCS24J_11403_15:54:05'

fig = go.Figure(go.Scattermapbox())
fig.update_layout(
    mapbox_style="carto-positron",
    showlegend=False,
    height=800,
    width=800,
    margin={"l": 0, "b": 0, "t": 0, "r": 0},
    # mapbox_center_lat=self.lat_cen,
    # mapbox_center_lon=self.lon_cen,
    mapbox_zoom=4,
)

fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=df_all[df_all.flight_id == id1].latitude,
        lon=df_all[df_all.flight_id == id1].longitude,
        text=df_all[df_all.flight_id == id1].timestamp,
        line=dict(width=2, color="black"),
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=df_all[df_all.flight_id == id2].latitude,
        lon=df_all[df_all.flight_id == id2].longitude,
        text=df_all[df_all.flight_id == id2].timestamp,
        line=dict(width=2, color="black"),
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=df_cube[df_cube.flight_id == id1].latitude,
        lon=df_cube[df_cube.flight_id == id1].longitude,
        text=df_cube[df_cube.flight_id == id1].timestamp,
        line=dict(width=2, color="red"),
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=df_cube[df_cube.flight_id == id2].latitude,
        lon=df_cube[df_cube.flight_id == id2].longitude,
        text=df_cube[df_cube.flight_id == id2].timestamp,
        line=dict(width=2, color="red"),
    )
)
fig.show()

In [ ]:
# Check for overlapping intervals
mask = df["max"].values[:, np.newaxis] > df["min"].values[np.newaxis, :]
mask &= np.triu(np.ones_like(mask), 1)

# Get the flight ids for overlapping intervals
i, j = np.nonzero(mask)
pairs = [(df["flight_id"][i[k]], df["flight_id"][j[k]]) for k in range(len(i))]
count = len(pairs)

In [ ]:
for cube in tqdm(lsaguac.cubes):
    subset = df_traf.loc[(df_traf['latitude'] >= cube.lat_min) & 
                     (df_traf['latitude'] <= cube.lat_max) &
                     (df_traf['longitude'] >= cube.lon_min) &
                     (df_traf['longitude'] <= cube.lon_max) &
                     (df_traf['altitude'] >= cube.alt_low) &
                     (df_traf['altitude'] <= cube.alt_high)]
    subset.to_parquet(f"../data/LSAGUAC/06_cube_data/{cube.id}_trajs.parquet", index=False)

    in_out = (
        subset.groupby("flight_id")["timestamp"]
        .agg(["min", "max"])
        .reset_index()
        .sort_values(by="min")
        )
    in_out.to_parquet(f"../data/LSAGUAC/06_cube_data/{cube.id}_inout.parquet", index=False)

    df = in_out

    count = 0
    pairs = []

    for i in range(len(df)):

        matches = []
        for j in range(i+1, len(df)):
            # Check for overlapping intervals
            if (df['max'][i] > df['min'][j]) and (df['max'][j] > df['min'][i]):
                count += 1
                matches.append(df['flight_id'][j])

        if len(matches) > 0:
            matches.append(df['flight_id'][i])
            pairs.append(tuple(matches))

    print(f'There were {len(df)} flights with {count} overlapping intervals.')

In [ ]:
lsaguac.cubes[0].alt_high

In [ ]:
from traffic.core import Traffic

trajs_low = Traffic.from_file("../data/LSAGUAC/05_low_traffic/trajs_tma_low.parquet")
trajs_low

In [ ]:
trajs_low = (
    trajs_low.resample("1s")
    .eval(desc="resampling", max_workers=20)
    )

In [ ]:
lsaguac.create_training_data()

In [ ]:
days = 100
freq = 120


seconds = days * 24 * 60 * 60

for i in range(seconds)

In [ ]:
import numpy as np

td = np.load('../data/LSAGUAC/06_training/X_norm.npy', allow_pickle=True)
td = td.reshape(td.shape + (1,))

In [ ]:
from complexity.vae import VAE

In [ ]:
autoencoder = VAE(
    input_shape=(100, 5, 1),
    conv_filters=[218, 12, 64],
    conv_kernels=[3, 3, 3],
    conv_strides=[1, 1, 1],
    latent_space_dim=8
)

In [ ]:
autoencoder.compile()

In [ ]:
for i in range(10000):
    autoencoder.train(td, 32, 1)
    autoencoder.save(save_folder='../data/LSAGUAC/07_model/model_test_1')

In [ ]:
recondst, latent = autoencoder.reconstruct(td)

In [ ]:
# read a txt file, separated by spaces
with open('../data/LSAGUAC/06_training/normalisation.txt', 'r') as f:
    data = f.readlines()[0]

data = data.split(' ')
data = [float(i) for i in data]
lat_max = data[0]
lat_min = data[1]
lon_max = data[2]
lon_min = data[3]
alt_max = data[4]
alt_min = data[5]
gs_max = data[6]
gs_min = data[7]
tm_max = data[8]
tm_min = data[9]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
x = 5

lat_orig = td[x,:,0,:]
lon_orig = td[x,:,1,:]
alt_orig = td[x,:,2,:]
gs_orig = td[x,:,3,:]
tm_orig = td[x,:,4,:]

lat_rec = recondst[x,:,0,:]
lon_rec = recondst[x,:,1,:]
alt_rec = recondst[x,:,2,:]
gs_rec = recondst[x,:,3,:]
tm_rec = recondst[x,:,4,:]

import matplotlib.pyplot as plt
plt.plot(lon_orig, lat_orig)
plt.plot(lon_rec, lat_rec)
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.show()

plt.plot(alt_orig)
plt.plot(alt_rec)
plt.ylim(-1,1)
plt.show()

plt.plot(gs_orig)
plt.plot(gs_rec)
plt.ylim(-1,1)
plt.show()

plt.plot(tm_orig)
plt.plot(tm_rec)
# plt.ylim(-1,1)
plt.show()

# Assuming your data is stored in a variable called "data"
pca = PCA(n_components=2)
data_reduced = pca.fit_transform(latent)

# Visualize the reduced data
plt.scatter(data_reduced[:,0], data_reduced[:,1], s=1)
plt.scatter(data_reduced[x,0], data_reduced[x,1], s=2, c='r')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

plt.scatter(latent[:,0], latent[:,5], s=1)
plt.scatter(latent[x,0], latent[x,5], s=1, c='r')
plt.xlabel('Latent Component 1')
plt.ylabel('Latent Component 2')
plt.show()

# tsne = TSNE(n_components=2)
# data_reduced = tsne.fit_transform(latent)
# plt.scatter(data_reduced[:,0], data_reduced[:,1], s=1)
# plt.scatter(data_reduced[x,0], data_reduced[x,1], s=1, c='r')
# plt.xlabel('t-SNE Component 1')
# plt.ylabel('t-SNE Component 2')
# plt.show()

In [ ]:
latent.shape

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN

# instantiate the DBSCAN algorithm
dbscan = DBSCAN(eps=0.14, min_samples=10)

# fit the model to the data
dbscan.fit(latent)

# retrieve the labels assigned to each point
labels = dbscan.labels_

# print the number of clusters found
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f'Number of clusters: {n_clusters}')

# print the labels assigned to each point
print(f'Labels: {labels}')

In [ ]:
# make a dataframe containing as columns the two principal components and labels
import pandas as pd
df = pd.DataFrame(data_reduced, columns=['PC1', 'PC2'])
df['labels'] = labels


In [ ]:
# make a scatterplot of the data colored by the assigned labels using seaborn
import seaborn as sns
sns.scatterplot(df, x='PC1', y='PC2', hue='labels', palette='Set1')
plt.show()